In [11]:
import tensorflow as tf
import os
from six.moves import cPickle
import collections
import numpy as np
import codecs
import urllib

### 读取和预处理数据

In [12]:
DATA_DIR = "./data/sherlock/"
DOWNLOAD_URL = "https://sherlock-holm.es/stories/plain-text/cnus.txt"

def make_dir(path):
    try:
        os.mkdir(path)
    except OSError:
        pass

make_dir("./data")
make_dir(DATA_DIR)

def download(file_path):
    if os.path.exists(file_path):
        print("Dataset ready")
        return file_path
    file_name, _ = urllib.request.urlretrieve(DOWNLOAD_URL, file_path)
    return file_path

file_path = download(os.path.join(DATA_DIR, "input.txt"))



input_file = os.path.join(DATA_DIR, "input.txt")
vocab_file = os.path.join(DATA_DIR, "vocab.pkl")
tensor_file = os.path.join(DATA_DIR, "data.npy")
    

with codecs.open(input_file, "r", encoding="utf-8") as f:
    data = f.read()
counter = collections.Counter(data)
counter_pairs = sorted(counter.items(), key=lambda x: -x[1])
chars, _ = zip(*counter_pairs)
vocab_size = len(chars)
vocab = dict(zip(chars, range(len(chars))))
tensor = np.array(list(map(vocab.get, data)))
    

Dataset ready


### 把数据处理成batch

In [7]:
batch_size = 50
seq_length = 50
num_batches = int(tensor.size / (batch_size * seq_length))
if num_batches == 0:
    assert False, "Not enough data. Make seq_length and batch_size small."
tensor = tensor[:num_batches * batch_size * seq_length]
xdata = tensor
ydata = np.copy(tensor)
ydata[:-1] = xdata[1:]
ydata[-1] = xdata[0]
x_batches = np.split(xdata.reshape(batch_size, -1), num_batches, 1)
y_batches = np.split(ydata.reshape(batch_size, -1), num_batches, 1)
pointer = 0

### 定义RNN模型

In [8]:
from tensorflow.contrib import rnn
from tensorflow.contrib import legacy_seq2seq


class Model():
    def __init__(self, training=True):
        self.batch_size = 50
        self.seq_length = 50
        if not training:
            self.batch_size = 1
            self.seq_length = 1
            
        self.rnn_size = 128
        self.num_layers = 2
        self.input_keep_prob = 1.0
        self.output_keep_prob = 1.0
        self.grad_clip = 5.0
        self.training = 1

        with tf.device("/gpu:0"):
            cell_fn = rnn.BasicRNNCell

            cells = []
            for _ in range(self.num_layers):
                cell = cell_fn(self.rnn_size)
                # dropout
                if training and (self.input_keep_prob < 1.0 or self.output_keep_prob < 1.0):
                    cell = rnn.DropoutWrapper(cell, input_keep_prob=self.input_keep_prob,
                                             output_keep_prob=self.output_keep_prob)
                cells.append(cell)

            self.cell = cell = rnn.MultiRNNCell(cells, state_is_tuple=True)

            # placeholder for input and output
            self.input_data = tf.placeholder(tf.int32, [self.batch_size, self.seq_length])
            self.output_data = tf.placeholder(tf.int32, [self.batch_size, self.seq_length])
            self.initial_state = cell.zero_state(self.batch_size, tf.float32)
            with tf.variable_scope("rnnlm"):
                softmax_w = tf.get_variable("softmax_w", [self.rnn_size, vocab_size])
                softmax_b = tf.get_variable("softmax_b", [vocab_size])
            embedding = tf.get_variable("embedding", [vocab_size, self.rnn_size])
            inputs = tf.nn.embedding_lookup(embedding, self.input_data)
            if training and self.output_keep_prob:
                inputs = tf.nn.dropout(inputs, self.output_keep_prob)

            inputs = tf.split(inputs, self.seq_length, 1) # returns self.seq_length Tensor objects
            inputs = [tf.squeeze(input_, [1]) for input_ in inputs] # squeeze the first dimension, got [B * embed_size] * self.seq_length Tensor objects

            def loop(prev, _):
                prev = tf.matmul(prev, softmax_w) + softmax_b
                prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
                return tf.nn.embedding_lookup(embedding, prev_symbol)

            # run an rnn on the sequence
            outputs, last_state = legacy_seq2seq.rnn_decoder(inputs, self.initial_state, cell, 
                                                             loop_function=loop if not training else None, scope="rnnlm")
            output = tf.reshape(tf.concat(outputs, 1), [-1, self.rnn_size])
            self.logits = tf.matmul(output, softmax_w) + softmax_b
            self.probs = tf.nn.softmax(self.logits)

            # loss over the whole sequence
            loss = legacy_seq2seq.sequence_loss_by_example([self.logits],
                                                          [tf.reshape(self.output_data, [-1])],
                                                          [tf.ones([self.batch_size * self.seq_length])])
            with tf.name_scope("cost"):
                self.cost = tf.reduce_sum(loss) / batch_size / seq_length
            self.final_state = last_state
            self.lr = tf.Variable(0.0, trainable=False)
            # gradient clipping on trainable variables
            tvars = tf.trainable_variables()
            grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), self.grad_clip)

            # optimizer and train
            with tf.name_scope("optimizer"):
                optimizer = tf.train.AdamOptimizer(self.lr)
            self.train_op = optimizer.apply_gradients(zip(grads, tvars))
    
    def sample(self, sess, chars, vocab, num=200, prime="The "):
        state = sess.run(self.cell.zero_state(1, tf.float32))
        for char in prime[:-1]:
            x = np.zeros((1,1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state: state}
            [state] = sess.run([self.final_state], feed)

        def weighted_pick(weights):
            t = np.cumsum(weights)
            s = np.sum(weights)
            return (int(np.searchsorted(t, np.random.rand(1)*s)))

        ret = prime
        char = prime[-1]
        for n in range(num):
            x = np.zeros((1,1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state: state}
            [p, state] = sess.run([self.probs, self.final_state], feed)
            p = p[0]
            sample = weighted_pick(p)

            pred = chars[sample]
            ret += pred
            char = pred
        return ret



### 开始训练RNN

In [9]:
import time
num_epochs = 10
learning_rate = 0.002
decay_rate = 0.97
save_dir = "./save/"
save_every = 500


tf.reset_default_graph()
model = Model() 


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())   
    saver = tf.train.Saver(tf.global_variables())

    for e in range(num_epochs):
        sess.run(tf.assign(model.lr, learning_rate*(decay_rate ** e)))
        # reset the pointer to load from the beginning
        pointer = 0
        state = sess.run(model.initial_state)
        for b in range(num_batches):
            start = time.time()
            x, y = x_batches[pointer], y_batches[pointer]
            pointer += 1
            feed = {model.input_data: x, model.output_data: y}
            feed[model.initial_state] = state
                
            train_loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)
            end = time.time()
            print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch={:.3f}".format(
                e * num_batches + b, num_epochs * num_batches, e, train_loss, end-start))
            
            if ( e * num_batches + b ) % save_every == 0 or ( 
                e == num_epochs - 1 and b == num_batches - 1):
                checkpoint_path = os.path.join(save_dir, "model.ckpt")
                saver.save(sess, checkpoint_path, global_step=e*num_batches + b)
                print("model saved to {}".format(checkpoint_path))
                
        
        

0/13520 (epoch 0), train_loss = 4.639, time/batch=0.631
model saved to ./save/model.ckpt
1/13520 (epoch 0), train_loss = 4.493, time/batch=0.028
2/13520 (epoch 0), train_loss = 4.212, time/batch=0.025
3/13520 (epoch 0), train_loss = 3.801, time/batch=0.025
4/13520 (epoch 0), train_loss = 3.489, time/batch=0.026
5/13520 (epoch 0), train_loss = 3.394, time/batch=0.026
6/13520 (epoch 0), train_loss = 3.252, time/batch=0.025
7/13520 (epoch 0), train_loss = 3.190, time/batch=0.025
8/13520 (epoch 0), train_loss = 3.124, time/batch=0.026
9/13520 (epoch 0), train_loss = 3.050, time/batch=0.026
10/13520 (epoch 0), train_loss = 2.996, time/batch=0.026
11/13520 (epoch 0), train_loss = 3.003, time/batch=0.026
12/13520 (epoch 0), train_loss = 3.018, time/batch=0.024
13/13520 (epoch 0), train_loss = 3.008, time/batch=0.024
14/13520 (epoch 0), train_loss = 3.013, time/batch=0.023
15/13520 (epoch 0), train_loss = 2.963, time/batch=0.024
16/13520 (epoch 0), train_loss = 3.003, time/batch=0.024
17/13520

144/13520 (epoch 0), train_loss = 2.028, time/batch=0.017
145/13520 (epoch 0), train_loss = 2.080, time/batch=0.018
146/13520 (epoch 0), train_loss = 2.019, time/batch=0.018
147/13520 (epoch 0), train_loss = 2.040, time/batch=0.017
148/13520 (epoch 0), train_loss = 2.070, time/batch=0.017
149/13520 (epoch 0), train_loss = 1.960, time/batch=0.017
150/13520 (epoch 0), train_loss = 2.038, time/batch=0.018
151/13520 (epoch 0), train_loss = 2.001, time/batch=0.018
152/13520 (epoch 0), train_loss = 1.985, time/batch=0.018
153/13520 (epoch 0), train_loss = 2.050, time/batch=0.018
154/13520 (epoch 0), train_loss = 1.955, time/batch=0.018
155/13520 (epoch 0), train_loss = 2.040, time/batch=0.018
156/13520 (epoch 0), train_loss = 2.055, time/batch=0.018
157/13520 (epoch 0), train_loss = 2.037, time/batch=0.022
158/13520 (epoch 0), train_loss = 1.961, time/batch=0.020
159/13520 (epoch 0), train_loss = 2.022, time/batch=0.017
160/13520 (epoch 0), train_loss = 1.992, time/batch=0.018
161/13520 (epo

297/13520 (epoch 0), train_loss = 1.795, time/batch=0.020
298/13520 (epoch 0), train_loss = 1.783, time/batch=0.019
299/13520 (epoch 0), train_loss = 1.851, time/batch=0.017
300/13520 (epoch 0), train_loss = 1.804, time/batch=0.018
301/13520 (epoch 0), train_loss = 1.837, time/batch=0.018
302/13520 (epoch 0), train_loss = 1.813, time/batch=0.018
303/13520 (epoch 0), train_loss = 1.791, time/batch=0.019
304/13520 (epoch 0), train_loss = 1.830, time/batch=0.018
305/13520 (epoch 0), train_loss = 1.834, time/batch=0.018
306/13520 (epoch 0), train_loss = 1.785, time/batch=0.018
307/13520 (epoch 0), train_loss = 1.784, time/batch=0.017
308/13520 (epoch 0), train_loss = 1.838, time/batch=0.018
309/13520 (epoch 0), train_loss = 1.815, time/batch=0.017
310/13520 (epoch 0), train_loss = 1.782, time/batch=0.017
311/13520 (epoch 0), train_loss = 1.807, time/batch=0.018
312/13520 (epoch 0), train_loss = 1.747, time/batch=0.019
313/13520 (epoch 0), train_loss = 1.785, time/batch=0.019
314/13520 (epo

448/13520 (epoch 0), train_loss = 1.656, time/batch=0.018
449/13520 (epoch 0), train_loss = 1.672, time/batch=0.017
450/13520 (epoch 0), train_loss = 1.643, time/batch=0.018
451/13520 (epoch 0), train_loss = 1.803, time/batch=0.018
452/13520 (epoch 0), train_loss = 1.735, time/batch=0.017
453/13520 (epoch 0), train_loss = 1.644, time/batch=0.018
454/13520 (epoch 0), train_loss = 1.639, time/batch=0.017
455/13520 (epoch 0), train_loss = 1.688, time/batch=0.019
456/13520 (epoch 0), train_loss = 1.711, time/batch=0.019
457/13520 (epoch 0), train_loss = 1.715, time/batch=0.018
458/13520 (epoch 0), train_loss = 1.728, time/batch=0.018
459/13520 (epoch 0), train_loss = 1.722, time/batch=0.018
460/13520 (epoch 0), train_loss = 1.702, time/batch=0.018
461/13520 (epoch 0), train_loss = 1.627, time/batch=0.018
462/13520 (epoch 0), train_loss = 1.703, time/batch=0.019
463/13520 (epoch 0), train_loss = 1.713, time/batch=0.018
464/13520 (epoch 0), train_loss = 1.653, time/batch=0.018
465/13520 (epo

596/13520 (epoch 0), train_loss = 1.596, time/batch=0.018
597/13520 (epoch 0), train_loss = 1.584, time/batch=0.018
598/13520 (epoch 0), train_loss = 1.578, time/batch=0.018
599/13520 (epoch 0), train_loss = 1.585, time/batch=0.018
600/13520 (epoch 0), train_loss = 1.643, time/batch=0.018
601/13520 (epoch 0), train_loss = 1.604, time/batch=0.018
602/13520 (epoch 0), train_loss = 1.596, time/batch=0.018
603/13520 (epoch 0), train_loss = 1.623, time/batch=0.018
604/13520 (epoch 0), train_loss = 1.652, time/batch=0.018
605/13520 (epoch 0), train_loss = 1.575, time/batch=0.018
606/13520 (epoch 0), train_loss = 1.679, time/batch=0.018
607/13520 (epoch 0), train_loss = 1.580, time/batch=0.018
608/13520 (epoch 0), train_loss = 1.640, time/batch=0.018
609/13520 (epoch 0), train_loss = 1.611, time/batch=0.018
610/13520 (epoch 0), train_loss = 1.572, time/batch=0.018
611/13520 (epoch 0), train_loss = 1.674, time/batch=0.018
612/13520 (epoch 0), train_loss = 1.630, time/batch=0.019
613/13520 (epo

...

12903/13520 (epoch 9), train_loss = 1.195, time/batch=0.019
12904/13520 (epoch 9), train_loss = 1.298, time/batch=0.017
12905/13520 (epoch 9), train_loss = 1.215, time/batch=0.018
12906/13520 (epoch 9), train_loss = 1.185, time/batch=0.018
12907/13520 (epoch 9), train_loss = 1.258, time/batch=0.018
12908/13520 (epoch 9), train_loss = 1.173, time/batch=0.018
12909/13520 (epoch 9), train_loss = 1.202, time/batch=0.017
12910/13520 (epoch 9), train_loss = 1.178, time/batch=0.018
12911/13520 (epoch 9), train_loss = 1.215, time/batch=0.018
12912/13520 (epoch 9), train_loss = 1.249, time/batch=0.018
12913/13520 (epoch 9), train_loss = 1.259, time/batch=0.018
12914/13520 (epoch 9), train_loss = 1.216, time/batch=0.018
12915/13520 (epoch 9), train_loss = 1.231, time/batch=0.018
12916/13520 (epoch 9), train_loss = 1.242, time/batch=0.018
12917/13520 (epoch 9), train_loss = 1.215, time/batch=0.018
12918/13520 (epoch 9), train_loss = 1.166, time/batch=0.018
12919/13520 (epoch 9), train_loss = 1.21

13047/13520 (epoch 9), train_loss = 1.150, time/batch=0.018
13048/13520 (epoch 9), train_loss = 1.196, time/batch=0.018
13049/13520 (epoch 9), train_loss = 1.211, time/batch=0.018
13050/13520 (epoch 9), train_loss = 1.226, time/batch=0.018
13051/13520 (epoch 9), train_loss = 1.248, time/batch=0.018
13052/13520 (epoch 9), train_loss = 1.208, time/batch=0.018
13053/13520 (epoch 9), train_loss = 1.236, time/batch=0.018
13054/13520 (epoch 9), train_loss = 1.210, time/batch=0.019
13055/13520 (epoch 9), train_loss = 1.179, time/batch=0.018
13056/13520 (epoch 9), train_loss = 1.302, time/batch=0.018
13057/13520 (epoch 9), train_loss = 1.253, time/batch=0.018
13058/13520 (epoch 9), train_loss = 1.298, time/batch=0.017
13059/13520 (epoch 9), train_loss = 1.289, time/batch=0.019
13060/13520 (epoch 9), train_loss = 1.255, time/batch=0.018
13061/13520 (epoch 9), train_loss = 1.226, time/batch=0.018
13062/13520 (epoch 9), train_loss = 1.262, time/batch=0.018
13063/13520 (epoch 9), train_loss = 1.23

13191/13520 (epoch 9), train_loss = 1.210, time/batch=0.018
13192/13520 (epoch 9), train_loss = 1.222, time/batch=0.019
13193/13520 (epoch 9), train_loss = 1.156, time/batch=0.019
13194/13520 (epoch 9), train_loss = 1.264, time/batch=0.018
13195/13520 (epoch 9), train_loss = 1.172, time/batch=0.017
13196/13520 (epoch 9), train_loss = 1.260, time/batch=0.018
13197/13520 (epoch 9), train_loss = 1.247, time/batch=0.018
13198/13520 (epoch 9), train_loss = 1.255, time/batch=0.018
13199/13520 (epoch 9), train_loss = 1.280, time/batch=0.018
13200/13520 (epoch 9), train_loss = 1.237, time/batch=0.017
13201/13520 (epoch 9), train_loss = 1.185, time/batch=0.018
13202/13520 (epoch 9), train_loss = 1.229, time/batch=0.018
13203/13520 (epoch 9), train_loss = 1.208, time/batch=0.017
13204/13520 (epoch 9), train_loss = 1.199, time/batch=0.018
13205/13520 (epoch 9), train_loss = 1.178, time/batch=0.019
13206/13520 (epoch 9), train_loss = 1.223, time/batch=0.018
13207/13520 (epoch 9), train_loss = 1.20

13332/13520 (epoch 9), train_loss = 1.242, time/batch=0.019
13333/13520 (epoch 9), train_loss = 1.192, time/batch=0.019
13334/13520 (epoch 9), train_loss = 1.203, time/batch=0.018
13335/13520 (epoch 9), train_loss = 1.227, time/batch=0.018
13336/13520 (epoch 9), train_loss = 1.255, time/batch=0.017
13337/13520 (epoch 9), train_loss = 1.157, time/batch=0.018
13338/13520 (epoch 9), train_loss = 1.194, time/batch=0.017
13339/13520 (epoch 9), train_loss = 1.200, time/batch=0.017
13340/13520 (epoch 9), train_loss = 1.211, time/batch=0.018
13341/13520 (epoch 9), train_loss = 1.205, time/batch=0.018
13342/13520 (epoch 9), train_loss = 1.196, time/batch=0.017
13343/13520 (epoch 9), train_loss = 1.233, time/batch=0.018
13344/13520 (epoch 9), train_loss = 1.234, time/batch=0.017
13345/13520 (epoch 9), train_loss = 1.239, time/batch=0.018
13346/13520 (epoch 9), train_loss = 1.266, time/batch=0.018
13347/13520 (epoch 9), train_loss = 1.325, time/batch=0.018
13348/13520 (epoch 9), train_loss = 1.28

13476/13520 (epoch 9), train_loss = 1.246, time/batch=0.018
13477/13520 (epoch 9), train_loss = 1.226, time/batch=0.018
13478/13520 (epoch 9), train_loss = 1.186, time/batch=0.018
13479/13520 (epoch 9), train_loss = 1.220, time/batch=0.018
13480/13520 (epoch 9), train_loss = 1.230, time/batch=0.018
13481/13520 (epoch 9), train_loss = 1.285, time/batch=0.017
13482/13520 (epoch 9), train_loss = 1.285, time/batch=0.018
13483/13520 (epoch 9), train_loss = 1.240, time/batch=0.017
13484/13520 (epoch 9), train_loss = 1.242, time/batch=0.018
13485/13520 (epoch 9), train_loss = 1.278, time/batch=0.018
13486/13520 (epoch 9), train_loss = 1.242, time/batch=0.018
13487/13520 (epoch 9), train_loss = 1.226, time/batch=0.018
13488/13520 (epoch 9), train_loss = 1.233, time/batch=0.018
13489/13520 (epoch 9), train_loss = 1.219, time/batch=0.018
13490/13520 (epoch 9), train_loss = 1.191, time/batch=0.018
13491/13520 (epoch 9), train_loss = 1.243, time/batch=0.018
13492/13520 (epoch 9), train_loss = 1.28

### 产生一些句子试一试

In [10]:
n = 500
prime = " "

tf.reset_default_graph()
model = Model(training=False)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state(save_dir)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print(model.sample(sess, chars, vocab, n, prime).encode("utf-8"))


# with open(os.path.join(save_dir, "config.pkl"), "rb") as f:
#     saved_args = cPickle.load(f)
# with open(os.path.join(save_dir, "chars_vocab.pkl"), "rb") as f:
#     chars, vocab = cPickle.load(f)


INFO:tensorflow:Restoring parameters from ./save/model.ckpt-13519
b' Holyes, who should alight of them, this own. A point-counterly at unalson\n     loftion my dear Hopkint than\n     the other liid. There is well times Gow I will do again you once attackfiest.\n\n     "Jowemon I heard them.\n\n     "Because.\n\n     Should that\n     could tongind it. "The man to the whee-told him, "I would swaving with how I may now it over It lay through\n     his heads, to accoUlf effect. I kidne within the time,\' silver lugtary to be held that  what danger. On not the micked, then?" t'


### Homework
- Implement char-rnn using LSTM and GRU
- Find your own dataset (例如找一个中文dataset, be creative on what dataset to use) and apply the same model, report your generated samples. 